# Contents
* Generate scenarios for the CA grid mix based off of 2016 - in particular we will use scenario 3X to represent 2025

In [ ]:
%matplotlib inline

import sys
import os

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import plotly
import plotly.graph_objs as go 
plotly.offline.init_notebook_mode()
GRAPH_CONFIG = {'showLink': False, 'displaylogo': False,                         
                'modeBarButtonsToRemove':['sendDataToCloud']}
from plotly.colors import DEFAULT_PLOTLY_COLORS

In [ ]:
pthNm = "../data/CAISO_DailyRenewablesWatch"
df = pd.read_csv(os.path.join(pthNm, 'DailyRenewablesWatch_2016.csv'), index_col=0,
                 parse_dates=True)

cols = [col for col in df.columns if col != 'carbon']
df["total"] = df[cols].sum(axis=1)
CARBON_INTENSITY = {"biogas":18, "biomass":18, "geo":42, "hydro":4,
                    "imports":428, "nuclear":16, "smhydro":4,
                    "solarpv":46, "solarth":22, "thermal":469, "wind":12}

df["carbon"] = df.apply(lambda row: sum(row[fuel]*CARBON_INTENSITY[fuel]
                                       for fuel in CARBON_INTENSITY)/1e3, axis=1)

In [ ]:
df.head()

In [ ]:
# Initial grid mix
tot=0
for col in cols:
    prct = 100*df[col].sum()/df["total"].sum()
    print("%s: %.2f"%(col,prct))
    tot+=prct
print("total: %.2f"%tot)

In [ ]:
# generate scenarios
frames=dict()
for x in [0,1,2,4,6]:
    print("\n\n\n",x)
    print("max overgen power on grid (GW)")
    print(min(df.total - (1+x) * df.solarpv)/1e3)

    df_solar = df.copy()

    # reduce thermal, but don't go negative
    df_solar.thermal = df_solar.apply(
        lambda row : max(row.thermal - x*row.solarpv, 0), axis=1)

    # take all the solar
    df_solar.solarpv = df_solar.apply(
        lambda row : (1 + x) * row.solarpv, axis=1)

    overgenP = df_solar.total - df_solar[cols].sum(axis=1)
    overgen = overgenP.sum()

    print("overgen energy %")
    print(100 * overgen/df_solar.total.sum())
    print(overgen)
    # reduce imports if needed
    if overgen<-1*1e5:
        print("Reducing imports as well")
        df_solar.imports = (df_solar.imports + overgenP)
        df_solar.imports = df_solar.apply(
            lambda row: max(row.imports, 0), axis=1)
        overgen = (df_solar.total - df_solar[cols].sum(axis=1)).sum()


    # reshape the gas/imports homogeneously to accomodate
    print("overgen energy %")
    print(100 * overgen/df_solar.total.sum())
    print(overgen)
    it=0
    while overgen<-1*1e3: # GWh
        it+=1
        if it%2==0:
            df_solar.thermal = df_solar.thermal.apply(
                lambda x:max(x+overgen/len(df_solar),0))
        else:
            df_solar.imports = df_solar.imports.apply(
                lambda x:max(x+overgen/len(df_solar),0))
        overgen = (df_solar.total - df_solar[cols].sum(axis=1)).sum()
        if sum(df_solar.thermal)<1e3 and sum(df_solar.imports)<1e-3:
            print("Too much solar!")

    print("there is %.2f overgen after %d iterations"%(overgen,it))

    # recalculate carbon intensity and total carbon of the grid
    df_solar["carbon"] = df_solar.apply(
        lambda row:sum(row[fuel]*CARBON_INTENSITY[fuel]
                       for fuel in CARBON_INTENSITY)/1e3, axis=1)
    df_solar["carbon_intensity"] = df_solar.apply(
        lambda row:row["carbon"]*1e3/row["total"], axis=1)

    # print summary stats
    print("old total gen (TWh)")
    print(df.total.sum()/1e6)
    print("new total gen (TWh)")
    print(df_solar.total.sum()/1e6)
    print("new total gen (TWh) - sanity")
    print(df_solar[cols].sum(axis=1).sum()/1e6)
    print("solar energy %")
    print(100*df_solar.solarpv.sum()/df_solar.total.sum())
    print("gas energy %")
    print(100*df_solar.thermal.sum()/df_solar.total.sum())
    print("imports %")
    print(100*df_solar.imports.sum()/df_solar.total.sum())
    #print("max solar GW (old)")
    #print(max(df_watttime.fuel_solar)/1e3)
    print("max solar (GW)")
    print(max(df_solar.solarpv)/1e3)
    print("Solar Gen (TWh)")
    print(df_solar.solarpv.sum()/1e6)
    print("CO2 emissions (Mtonnes)")
    print(df_solar.carbon.sum()/1e6)
    print("Yearly CO2 emissions reduction (%)")
    print(100*(1-df_solar.carbon.sum()/df.carbon.sum()))
    
    frames[x] = df_solar

# Plot monthly emissions
For these scenarios and compare to what I downloaded from the CAISO website. Note that I use LCA numbers to compute emissions from the grid mix. CAISO may be using direct emissions estimates

In [ ]:
graycols=['rgb(200,200,220)','rgb(150,150,150)','rgb(100,100,100)',
          None,'rgb(50,50,50)']

df_hist = pd.read_csv(os.path.join(pthNm, 'CAISO-historical-co2.csv'))
cols=df_hist.columns[1:]

plot_data = []

df["month"] = df.index.month

for x in frames:
    df_solar = frames[x]
    df_solar["month"] = df_solar.index.month
    lineLab = "x"+str(x+1)
    if x==0:
        lineLab+=" (2016)"
    plot_data += [go.Scatter(name = lineLab,
                      x=cols,
                      y=df_solar.loc[:, ["month", "carbon"]]\
                             .groupby("month").sum().values.flatten()/1e6
                     )]

# make subplot to show historical data
for i, year in enumerate(range(2013,2018)):
    if i==3:
        color=DEFAULT_PLOTLY_COLORS[0]
    else:
        color = graycols[i]
            
    dfp = pd.read_csv(pthNm+'/DailyRenewablesWatch_' + str(year)+'.csv',
                      index_col=0, parse_dates=True)
    dfp["month"] = dfp.index.month
    
    # recalculate carbon
    dfp["carbon"] = dfp.apply(
        lambda row:sum(row[fuel]*CARBON_INTENSITY[fuel]
                       for fuel in CARBON_INTENSITY)/1e3, axis=1)
    plot_data += [go.Scatter(name = str(year),
                      x=cols,
                      xaxis='x2',
                      y=dfp.loc[:, ["month", "carbon"]]\
                             .groupby("month").sum().values.flatten()/1e6,
                      line = dict(color = color)
                     )]
    if i>0:
        
        plot_data += [go.Scatter(name = str(year)+'a',
                          x=cols,
                          xaxis='x2',       
                          y=df_hist[cols][df_hist.year==year]\
                                 .values[0]/1e6,
                          line = dict(dash='dash',
                                      color = color)
                         )]

layout = dict(
    font=dict(size=18),
    autosize=False,
    width=2*650,
    height=500,
    title='Monthly CO2 Emissions',
    xaxis=dict(domain=[0, 0.49]),
    xaxis2=dict(domain=[0.51, 1]),
    yaxis=dict(title='mTCO2 (millions)',
               range=[0,8]),
    annotations=[{'font': {'size': 22},
                      'showarrow': False,
                      'text': 'Scenarios',
                      'x': 0.225,
                      'xanchor': 'center',
                      'xref': 'paper',
                      'y': 1.0,
                       'yanchor': 'bottom',
                       'yref': 'paper'},
                      {'font': {'size': 22},
                       'showarrow': False,
                       'text': 'Historical',
                       'x': 0.775,
                       'xanchor': 'center',
                       'xref': 'paper',
                       'y': 1.0,
                       'yanchor': 'bottom',
                       'yref': 'paper'}])

fig = go.Figure(data=plot_data, layout=layout)
plotly.offline.iplot(fig, config=GRAPH_CONFIG)

In [ ]:
for x in frames:
    df_extract = frames[x].loc[:,['carbon_intensity']]
    df_extract.to_csv("../data/carbon_scenarios/carbon_x%d.csv" % x)